In [25]:
import open3d
import numpy as np
import os
import utils.fread as fread
import utils.helpers as helpers
import utils.registration as registration
import utils.FCGF as FCGF
import copy
import tqdm

In [2]:
sequence_dir = "../localization-data/data/point_clouds/exp_5/trial_1/subject-1/01"
sequence_ts = fread.get_timstamps(sequence_dir, ext=".secondary.pcd")

num_frames = len(sequence_ts)

In [4]:
source = helpers.read_pcd(os.path.join(sequence_dir, f"{sequence_ts[0]}.secondary.pcd"))
target = helpers.read_pcd(os.path.join(sequence_dir, f"{sequence_ts[1]}.secondary.pcd"))

In [5]:
source = helpers.downsample(source, 0.05)
target = helpers.downsample(target, 0.05)

source_fpfh = registration.compute_fpfh(source, 0.05)
target_fpfh = registration.compute_fpfh(target, 0.05)

In [7]:
result = registration.ransac_feature_matching(source, target, source_fpfh, target_fpfh, 4, 0.05, p2p=False)

registration.describe(source, target, result)

In [9]:
registration.view(source, target, result.transformation)

Registering a sequence

In [29]:
local_pcds = []
fpfh_feats = []

for t in tqdm.trange(num_frames):
    pcd = helpers.read_pcd(os.path.join(sequence_dir, f"{sequence_ts[t]}.secondary.pcd"))
    pcd = helpers.downsample(pcd, 0.05)
    fpfh = registration.compute_fpfh(pcd, 0.05)
    local_pcds.append(pcd)
    fpfh_feats.append(fpfh) 

100%|██████████| 368/368 [00:08<00:00, 45.63it/s]


In [27]:
local_t = [np.identity(4)]

for t in tqdm.trange(num_frames - 1):
    source, source_fpfh = copy.deepcopy(local_pcds[t + 1]), fpfh_feats[t + 1]
    target, target_fpfh = copy.deepcopy(local_pcds[t]), fpfh_feats[t]

    ransac_reg = registration.ransac_feature_matching(source, target, source_fpfh, target_fpfh, n_ransac=4, threshold=0.05)
        
    local_t.append(ransac_reg.transformation)

100%|██████████| 367/367 [00:48<00:00,  7.59it/s]


In [30]:
trajectory_t = [np.identity(4)]

for t in tqdm.trange(num_frames):
    if t > 0:
        trajectory_t.append(np.dot(trajectory_t[t - 1], local_t[t]))
    local_pcds[t].transform(trajectory_t[t])

100%|██████████| 368/368 [00:00<00:00, 8806.03it/s]


In [31]:
open3d.visualization.draw_geometries(local_pcds)

In [3]:
local_pcds = []

for t in tqdm.trange(num_frames):
    pcd = helpers.read_pcd(os.path.join(sequence_dir, f"{sequence_ts[t]}.secondary.pcd"))
    pcd = helpers.downsample(pcd, 0.05)
    local_pcds.append(pcd)

100%|██████████| 368/368 [00:11<00:00, 31.24it/s]


In [4]:
fpfh_feats = [registration.compute_fpfh(local_pcds[0], 0.05)]
local_t = [np.identity(4)]

for t in tqdm.trange(num_frames - 1):
    source, source_fpfh = local_pcds[t + 1], registration.compute_fpfh(local_pcds[t + 1], 0.05)
    target, target_fpfh = local_pcds[t], fpfh_feats[t]

    ransac_reg = registration.ransac_feature_matching(source, target, source_fpfh, target_fpfh, n_ransac=4, threshold=0.05)
        
    local_t.append(ransac_reg.transformation)
    fpfh_feats.append(source_fpfh)

100%|██████████| 367/367 [00:59<00:00,  6.22it/s]


In [5]:
trajectory_t = [np.identity(4)]

for t in tqdm.trange(num_frames):
    if t > 0:
        trajectory_t.append(np.dot(trajectory_t[t - 1], local_t[t]))
    local_pcds[t].transform(trajectory_t[t])

100%|██████████| 368/368 [00:00<00:00, 10933.65it/s]


In [6]:
open3d.visualization.draw_geometries(local_pcds)

Testing groundtruth poses

In [33]:
features_dir = "data/features"
experiment = "exp_7"
trial = 1
subject = "subject-1"
sequence = 2
voxel_size = 0.03

sequence_dir = os.path.join(features_dir, experiment, f"trial_{trial}", str(voxel_size), subject, f"{sequence:02d}")
sequence_ts = fread.get_timstamps(sequence_dir, ext=".secondary.npz")

num_frames = len(sequence_ts)

file_name = f"{experiment}__trial_{trial}__{subject}__{sequence:02d}"

pose_file = os.path.join("data/trajectories/groundtruth", experiment, f"{file_name}.pose.npz")

In [34]:
poses = np.load(pose_file)

In [35]:
local_t = poses["local_t"]
trajectory_t = poses["trajectory_t"]

In [36]:
local_pcds = []

for t in tqdm.trange(num_frames):
    feature_file = os.path.join(sequence_dir, f"{sequence_ts[t]}.secondary.npz")
    pcd, _, _ = FCGF.get_features(feature_file)
    local_pcds.append(pcd)

100%|██████████| 395/395 [00:12<00:00, 31.65it/s]


In [37]:
for t in tqdm.trange(num_frames):
    local_pcds[t].transform(trajectory_t[t])

100%|██████████| 395/395 [00:00<00:00, 13530.23it/s]


In [38]:
trajectory_pcd = helpers.merge_pcds(local_pcds, 0.03)

In [39]:
open3d.visualization.draw_geometries([trajectory_pcd])

In [41]:
xyz = trajectory_t[:, :3, 3]

pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(xyz)

In [42]:
open3d.visualization.draw_geometries([pcd])

In [44]:
xyz[-1]

array([-7.31623922, -0.74977918,  5.76653422])